<a href="https://colab.research.google.com/github/Juhi-Purswani/Tensorflow_Multiclass_Image_Classification/blob/master/CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive') 


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [0]:
import os
os.chdir("./drive/My Drive/Plant_Seedlings")

In [4]:
import cv2
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.utils import shuffle

Using TensorFlow backend.


In [0]:
IMG_SIZE = 48
CHANNELS = 3
TRAIN_DIR = 'plant_seedlings'
#TEST_DIR = ''
batch_size = 128
learning_rate = 0.001
training_iters = 20
n_classes = 12

In [0]:
def create_features_labels(DIR):
  features = []
  labels = []
  dir_number = 0
  car_names = os.listdir(DIR)
  for name in car_names:
    path = DIR + '/' + name
    print("dir name and number" , path , 'and',dir_number)
    print(path)
    image_names = os.listdir(path)
    for image_name in image_names:
      image_path = path + '/' + image_name
      img = cv2.imread(image_path,cv2.IMREAD_COLOR)
      img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
      features.append(img)
      labels.append(np.array(dir_number))
    dir_number = dir_number + 1
  return(features , labels)

In [7]:
train_features,train_labels = create_features_labels(TRAIN_DIR)
#test_features,test_labels = create_features_labels(TEST_DIR)
train_features = np.asarray(train_features).reshape(-1,IMG_SIZE,IMG_SIZE,CHANNELS)
#test_features = np.asarray(test_features).reshape(-1,IMG_SIZE,IMG_SIZE,CHANNELS)

dir name and number plant_seedlings/Scentless Mayweed and 0
plant_seedlings/Scentless Mayweed
dir name and number plant_seedlings/Loose Silky-bent and 1
plant_seedlings/Loose Silky-bent
dir name and number plant_seedlings/Common wheat and 2
plant_seedlings/Common wheat
dir name and number plant_seedlings/Charlock and 3
plant_seedlings/Charlock
dir name and number plant_seedlings/Sugar beet and 4
plant_seedlings/Sugar beet
dir name and number plant_seedlings/Small-flowered Cranesbill and 5
plant_seedlings/Small-flowered Cranesbill
dir name and number plant_seedlings/Cleavers and 6
plant_seedlings/Cleavers
dir name and number plant_seedlings/Fat Hen and 7
plant_seedlings/Fat Hen
dir name and number plant_seedlings/Common Chickweed and 8
plant_seedlings/Common Chickweed
dir name and number plant_seedlings/Shepherd’s Purse and 9
plant_seedlings/Shepherd’s Purse
dir name and number plant_seedlings/Maize and 10
plant_seedlings/Maize
dir name and number plant_seedlings/Black-grass and 11
plan

In [0]:
features, labels = shuffle(train_features,train_labels)

In [0]:
labels = np.asarray(labels)
#test_labels = np.asarray(test_labels)
labels = to_categorical(labels)
#test_labels = to_categorical(test_labels)

In [10]:
print(features.shape,labels.shape)
#print(test_features.shape,test_labels.shape)

(5555, 48, 48, 3) (5555, 12)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

In [0]:
X_train = (X_train/np.float32(255))
X_test = X_test/np.float32(255)

In [13]:
x = tf.placeholder("float",[None,IMG_SIZE,IMG_SIZE,CHANNELS])
y = tf.placeholder("float",[None,n_classes])

conv1 = tf.layers.conv2d(inputs=x,filters=64,kernel_size=[3,3],padding="same",activation =tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],strides=2)
conv2 = tf.layers.conv2d(inputs=pool1,filters=32,kernel_size=[3,3],padding="same",activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],strides=2)
pool3_flat = tf.reshape(pool2,[-1,12*12*32])
dense1 = tf.layers.dense(inputs=pool3_flat,units=1000,activation=tf.nn.relu)
dense2 = tf.layers.dense(inputs=dense1,units=100,activation=tf.nn.relu)
out = tf.layers.dense(inputs=dense2,units=12,activation=tf.nn.relu)

W0626 20:04:14.653310 139812096583552 deprecation.py:323] From <ipython-input-13-1d7460cc66c5>:4: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0626 20:04:14.666306 139812096583552 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0626 20:04:15.097955 139812096583552 deprecation.py:323] From <ipython-input-13-1d7460cc66c5>:5: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0626 20:04:15.270048 139812096583552 deprecation.py:323] F

In [14]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out,labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
#Two more nodes for accuray evaluation
correct_prediction = tf.equal(tf.argmax(out,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
init = tf.global_variables_initializer()

W0626 20:04:17.093565 139812096583552 deprecation.py:323] From <ipython-input-14-827d028d1050>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [15]:
with tf.Session() as sess:
    sess.run(init)
    train_loss = []
    train_accuracy = []
    #summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    for i in range(training_iters):
        for batch in range(len(train_features)//batch_size):
            batch_x = X_train[batch*batch_size:min((batch+1)*batch_size,len(X_train))]
            batch_y = y_train[batch*batch_size:min((batch+1)*batch_size,len(y_train))]
            #print(batch_x.shape,batch_y.shape)
            a,b,c,d = batch_x.shape
            if(a==0):
              break
            opt = sess.run(optimizer, feed_dict={x: batch_x,y: batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y})
        print("the ", i , "th iteration with loss: " , loss, " and accuracy: ", acc)
        #print("_____________________________________________________________")
        #print(loss)
        #print("_____________________________________________________________")
        train_loss.append(loss)
        train_accuracy.append(acc)
    #summary_writer.close()

the  0 th iteration with loss:  2.2944124  and accuracy:  0.27173913
the  1 th iteration with loss:  2.0926876  and accuracy:  0.38043478
the  2 th iteration with loss:  1.758475  and accuracy:  0.4456522
the  3 th iteration with loss:  1.6105422  and accuracy:  0.5326087
the  4 th iteration with loss:  1.4820217  and accuracy:  0.59782606
the  5 th iteration with loss:  1.3846036  and accuracy:  0.6413044
the  6 th iteration with loss:  1.2686042  and accuracy:  0.6413044
the  7 th iteration with loss:  1.1702846  and accuracy:  0.6630435
the  8 th iteration with loss:  1.1028937  and accuracy:  0.6630435
the  9 th iteration with loss:  1.0932964  and accuracy:  0.6630435
the  10 th iteration with loss:  1.019954  and accuracy:  0.6847826
the  11 th iteration with loss:  0.98421794  and accuracy:  0.6956522
the  12 th iteration with loss:  0.94190794  and accuracy:  0.6956522
the  13 th iteration with loss:  0.92762136  and accuracy:  0.6956522
the  14 th iteration with loss:  0.93246